In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
import pandas as pd

In [2]:
labels = ["pushup_up",
        "pushup_down",
        "plank",
        "squat_up",
        "squat_down",
          "jumping_jack_start",
        "jumping_jack_end",
          "ab_crunch_down",
          "ab_crunch_up"]

In [3]:
model = load_model("model1.h5")
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 9)              │           297 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,435 (91.55 KB)

 Trainable params: 23,433 (91.54 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [5]:
# Inisialisasi MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# File input video (ganti dengan video yang ingin digunakan)
video_path = "./video/ab_crunch3.mp4"  # Ubah ini dengan path video kamu
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video selesai atau tidak bisa dibuka.")
        break
    
    # Konversi frame ke RGB (MediaPipe membutuhkan input RGB)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # Gambar landmark di video
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Test data jika ditekan 's'
        key = cv2.waitKey(0) & 0xFF  # Ganti dengan waitKey(1)

        if key == ord('a'):
            print("continue")

        if key == ord('s'):
            landmarks = results.pose_landmarks.landmark
            # Ekstrak koordinat x, y, z dari landmark
            # data = np.array([[lmk.x, lmk.y, lmk.z] for lmk in landmarks]).flatten()
            data = [landmark.x for landmark in landmarks] + \
                   [landmark.y for landmark in landmarks] + \
                   [landmark.z for landmark in landmarks]

            # Tambahkan dimensi batch ke data
            single_sample = np.expand_dims(data, axis=0)  # Bentuk: (1, num_features)

            # Prediksi pose menggunakan model MLP (tanpa normalisasi)
            y_pred_proba = model.predict(single_sample)  # Gunakan single_sample, bukan data
            y_pred = np.argmax(y_pred_proba, axis=1)
            class_label = labels[y_pred[0]]

            # Tampilkan hasil prediksi di frame
            cv2.putText(image, f"Pose: {class_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Tampilkan video
    cv2.imshow('Push-Up Data Collection (Video)', image)

    # Tekan 'q' untuk keluar
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("Selesai! Data push-up telah disimpan.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0